### 1. 引入大模型

In [179]:
from utils import get_ernie_models
from utils import get_qwen_models

In [4]:
llm_qwen, chat_qwen, embed_qwen = get_qwen_models()
llm_ernie, chat_ernie, embed_ernie = get_ernie_models()

### 2. 引入一段文本

In [5]:
from langchain_community.document_loaders import TextLoader

In [8]:
text_loader = TextLoader(file_path="./大语言模型.txt", encoding="utf8")

In [9]:
docs = text_loader.load()

### 3. 切分文本

In [10]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [11]:
spliter = RecursiveCharacterTextSplitter(chunk_size=128, chunk_overlap=64)

In [13]:
docs = spliter.split_documents(documents=docs)

### 4. 向量化

In [16]:
for idx, doc in enumerate(docs):
    print(idx, len(doc.page_content))

0 50
1 124
2 13
3 117
4 125
5 106
6 121
7 94
8 116
9 93
10 104
11 121
12 80


In [18]:
docs[0].page_content

'大语言模型作为人工智能领域的重要技术之一，具有广泛的应用场景。以下是十个方面的应用场景及其详细描述：'

In [25]:
from langchain_chroma import Chroma

In [26]:
# Chroma 基于 sqlite存储数据 

In [28]:
store = Chroma.from_documents(documents=docs, 
                              embedding=embed_ernie, 
                              persist_directory="chroma_data")

In [32]:
store.similarity_search(query="小米估值多少？", k=2)

[Document(metadata={'source': './大语言模型.txt'}, page_content='2. 智能客服与聊天机器人'),
 Document(metadata={'source': './大语言模型.txt'}, page_content='大语言模型作为人工智能领域的重要技术之一，具有广泛的应用场景。以下是十个方面的应用场景及其详细描述：')]

In [33]:
store.similarity_search_by_vector(embedding=embed_ernie.embed_query(text="小米估值多少？"), k=2)

[Document(metadata={'source': './大语言模型.txt'}, page_content='2. 智能客服与聊天机器人'),
 Document(metadata={'source': './大语言模型.txt'}, page_content='大语言模型作为人工智能领域的重要技术之一，具有广泛的应用场景。以下是十个方面的应用场景及其详细描述：')]

In [39]:
# 欧式距离度量（没有开方）
store.similarity_search_with_score(query="小米估值多少？", k=2)

[(Document(metadata={'source': './大语言模型.txt'}, page_content='2. 智能客服与聊天机器人'),
  0.6204638204734216),
 (Document(metadata={'source': './大语言模型.txt'}, page_content='大语言模型作为人工智能领域的重要技术之一，具有广泛的应用场景。以下是十个方面的应用场景及其详细描述：'),
  0.811250308581407)]

In [40]:
# 相关性转换
store.similarity_search_with_relevance_scores(query="小米估值多少？", k=2)

[(Document(metadata={'source': './大语言模型.txt'}, page_content='2. 智能客服与聊天机器人'),
  0.561265825062331),
 (Document(metadata={'source': './大语言模型.txt'}, page_content='大语言模型作为人工智能领域的重要技术之一，具有广泛的应用场景。以下是十个方面的应用场景及其详细描述：'),
  0.4263594055624079)]

In [41]:
import numpy as np

In [46]:
query = np.array(embed_ernie.embed_query(text="小米估值多少？"))
doc1 = np.array(embed_ernie.embed_query(text="2. 智能客服与聊天机器人"))
doc2 = np.array(embed_ernie.embed_query(text="大语言模型作为人工智能领域的重要技术之一，具有广泛的应用场景。以下是十个方面的应用场景及其详细描述："))

In [48]:
# 欧式距离
((query - doc1) ** 2).sum()

0.6204636182956037

In [49]:
# 欧式距离
((query - doc2) ** 2).sum()

0.8112505547683875

In [50]:
# 相关系数
1 - 0.6204636182956037 / 2 ** 0.5

0.561265968023637

In [51]:
# 相关系数
1 - 0.8112505547683875 / 2 ** 0.5

0.42635923148192456

$ relevance = 1 - \frac{distance}{\sqrt{2}}$

In [53]:
store.search(query="你好", search_type="similarity")

[Document(metadata={'source': './大语言模型.txt'}, page_content='2. 智能客服与聊天机器人'),
 Document(metadata={'source': './大语言模型.txt'}, page_content='大语言模型作为人工智能领域的重要技术之一，具有广泛的应用场景。以下是十个方面的应用场景及其详细描述：'),
 Document(metadata={'source': './大语言模型.txt'}, page_content='综上所述，大语言模型在多个领域都具有广泛的应用前景和巨大的价值潜力。随着技术的不断进步和完善，我们有理由相信大语言模型将在未来的人工智能领域发挥更加重要的作用。'),
 Document(metadata={'source': './大语言模型.txt'}, page_content='4. 情感分析\n描述：大语言模型通过分析文本中的情感倾向和情感表达，帮助企业了解客户反馈和情感状态，从而制定更精准的营销策略或优化客户服务。这种技术还可用于社交媒体监控，实时分析公众对某一主题或事件的情绪和反应。')]

In [54]:
import uuid

In [60]:
def get_doc_id():
    return str(uuid.uuid4()).replace("-", "")

In [135]:
ids = [get_doc_id() for _ in range(len(docs))]

In [136]:
docs

[Document(metadata={'source': './大语言模型.txt'}, page_content='大语言模型作为人工智能领域的重要技术之一，具有广泛的应用场景。以下是十个方面的应用场景及其详细描述：'),
 Document(metadata={'source': './大语言模型.txt'}, page_content='1. 机器翻译\n描述：大语言模型通过训练可以学习不同语言之间的语法和语义规则，实现自动翻译。这种技术已广泛应用于跨国企业沟通、国际合作等领域，如谷歌翻译等产品。尽管在处理长句子和歧义消解等方面仍面临挑战，但随着技术的发展，其准确性和流畅度不断提升。'),
 Document(metadata={'source': './大语言模型.txt'}, page_content='2. 智能客服与聊天机器人'),
 Document(metadata={'source': './大语言模型.txt'}, page_content='描述：大语言模型被用于开发智能客服助手和聊天机器人，能够理解用户的问题并提供相应的解决方案或转达给相关部门。这不仅提高了客服效率，还提升了用户体验。例如，通过自然语言处理技术，智能客服助手可以分析用户情感状态，及时发现问题并优化服务。'),
 Document(metadata={'source': './大语言模型.txt'}, page_content='3. 文本生成与创作\n描述：大语言模型能够生成符合语法规则的文章、新闻、小说等文本内容。通过学习大量文本数据，模型可以生成具有创造性和相关性的内容，广泛应用于新闻报道、广告营销等领域。此外，它还能根据给定主题或关键词生成文章，为创作者提供灵感和辅助。'),
 Document(metadata={'source': './大语言模型.txt'}, page_content='4. 情感分析\n描述：大语言模型通过分析文本中的情感倾向和情感表达，帮助企业了解客户反馈和情感状态，从而制定更精准的营销策略或优化客户服务。这种技术还可用于社交媒体监控，实时分析公众对某一主题或事件的情绪和反应。'),
 Document(metadata={'source': './大语言模型.txt'}, page_content='5. 自动问答系统\n描述：通过学习

In [137]:
ids

['812ddb6f17a14feb92727554a696067d',
 '3e9919bdba414cc5a63f78c011496b6d',
 'c920dde13480490daed97f77fdcbee43',
 'bd1b46d882c2483ea0ae703b40a282c9',
 'bae884e811794044afe0452da0f01008',
 '0bf8fda8ffca415e90cf1db04fbfdc51',
 '315803c708644833bdd0a908da184272',
 'c49698344d3a4b91b7bc3e9194ce17af',
 '36b94fcf040e4acd941e3528a01b8b39',
 '765e259e53714b568eaffc4dc9a23f47',
 '3e52b0de77a849d599eb4560506016b9',
 '207b82a9dcec4def8148653d183c3d16',
 '556bf536127e415287c3d9492ecd6944']

In [138]:
embed_ernie

QianfanEmbeddingsEndpoint(qianfan_ak=SecretStr('**********'), qianfan_sk=SecretStr('**********'), chunk_size=16, model='bge-large-zh', endpoint='', client=<qianfan.resources.llm.embedding.Embedding object at 0x000002D5C30FFDA0>, init_kwargs={}, model_kwargs={})

In [139]:
store1 = Chroma.from_documents(documents=docs, embedding=embed_ernie, ids=ids, persist_directory="chroma_data1")

In [140]:
store1.similarity_search_with_relevance_scores(query="你好吗？")

[(Document(metadata={'source': './大语言模型.txt'}, page_content='2. 智能客服与聊天机器人'),
  0.579608274172212),
 (Document(metadata={'source': './大语言模型.txt'}, page_content='大语言模型作为人工智能领域的重要技术之一，具有广泛的应用场景。以下是十个方面的应用场景及其详细描述：'),
  0.44239139980210307),
 (Document(metadata={'source': './大语言模型.txt'}, page_content='5. 自动问答系统\n描述：通过学习大量问题和答案，大语言模型能够自动生成符合语法规则的问题和答案。这种自动问答系统可应用于智能助手、搜索引擎等领域，为用户提供高效、准确的信息服务。结合知识图谱技术，问答系统的知识检索和推理能力得到进一步增强。'),
  0.4118836648456027),
 (Document(metadata={'source': './大语言模型.txt'}, page_content='综上所述，大语言模型在多个领域都具有广泛的应用前景和巨大的价值潜力。随着技术的不断进步和完善，我们有理由相信大语言模型将在未来的人工智能领域发挥更加重要的作用。'),
  0.4073137869770076)]

In [141]:
store1.search(query="你好", search_type="similarity")

[Document(metadata={'source': './大语言模型.txt'}, page_content='2. 智能客服与聊天机器人'),
 Document(metadata={'source': './大语言模型.txt'}, page_content='大语言模型作为人工智能领域的重要技术之一，具有广泛的应用场景。以下是十个方面的应用场景及其详细描述：'),
 Document(metadata={'source': './大语言模型.txt'}, page_content='综上所述，大语言模型在多个领域都具有广泛的应用前景和巨大的价值潜力。随着技术的不断进步和完善，我们有理由相信大语言模型将在未来的人工智能领域发挥更加重要的作用。'),
 Document(metadata={'source': './大语言模型.txt'}, page_content='4. 情感分析\n描述：大语言模型通过分析文本中的情感倾向和情感表达，帮助企业了解客户反馈和情感状态，从而制定更精准的营销策略或优化客户服务。这种技术还可用于社交媒体监控，实时分析公众对某一主题或事件的情绪和反应。')]

In [153]:
import chromadb
from chromadb import Settings

In [167]:
# 内存中 ./chroma
chroma_client = chromadb.Client()

In [168]:
collection = chroma_client.get_or_create_collection(name="my_collection")

In [161]:
collection.add(
    documents=["This is a document about engineer", "This is a document about steak"],
    metadatas=[{"source": "doc1"}, {"source": "doc2"}],
    ids=[get_doc_id(), get_doc_id()]
)

In [170]:
collection.count()

4

In [175]:
collection.query(query_texts=["scientist"], n_results=1)

{'ids': [['437f7dfe8cbc407b8e212120afc2329b']],
 'distances': [[1.2798278331756592]],
 'metadatas': [[{'source': 'doc1'}]],
 'embeddings': None,
 'documents': [['This is a document about engineer']],
 'uris': None,
 'data': None,
 'included': ['metadatas', 'documents', 'distances']}

In [177]:
collection.delete(ids=['437f7dfe8cbc407b8e212120afc2329b'])

In [178]:
collection.count()

3